In [ ]:
import carla
import random
import math
import time

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.load_world('Town03')
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter("vehicle.audi.tt")[0]
spawn_point = random.choice(world.get_map().get_spawn_points())
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
lidar_bp.set_attribute('range', '100')
lidar_transform = carla.Transform(carla.Location(x=0, z=2))
lidar_sensor = world.spawn_actor(lidar_bp, lidar_transform, attach_to=vehicle)

starting_point = (spawn_point.location.x, spawn_point.location.y)
end = (starting_point[0] + 50, starting_point[1] + 50)  # Example goal
map_size = [100, 100]
step_size = 5.0


In [ ]:
class Nodes:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.parent_x = []
        self.parent_y = []

def collision_check(x1, y1, x2, y2, obstacles):
    for obs in obstacles:
        obs_x, obs_y, size = obs
        dist = ((x1 - obs_x)**2 + (y1 - obs_y)**2)**0.5
        if dist < size:
            return True
    return False

def dist_and_angle(x1, y1, x2, y2):
    dist = math.sqrt(((x1 - x2)**2) + ((y1 - y2)**2))
    angle = math.atan2(y2 - y1, x2 - x1)
    return dist, angle

def nearest_node(x, y, node_list):
    temp_dist = []
    for node in node_list:
        dist, _ = dist_and_angle(x, y, node.x, node.y)
        temp_dist.append(dist)
    return temp_dist.index(min(temp_dist))

def rnd_point(map_size):
    new_x = random.uniform(0, map_size[0])
    new_y = random.uniform(0, map_size[1])
    return new_x, new_y

def RRT(carla_world, vehicle, start, end, step_size, map_size, lidar_data):
    node_list = [Nodes(start[0], start[1])]
    path_found = False

    while not path_found:
        nx, ny = rnd_point(map_size)
        nearest_idx = nearest_node(nx, ny, node_list)
        nearest_x = node_list[nearest_idx].x
        nearest_y = node_list[nearest_idx].y

        dist, theta = dist_and_angle(nx, ny, nearest_x, nearest_y)
        tx = nearest_x + step_size * math.cos(theta)
        ty = nearest_y + step_size * math.sin(theta)

        if not collision_check(tx, ty, nearest_x, nearest_y, lidar_data):
            new_node = Nodes(tx, ty)
            new_node.parent_x = node_list[nearest_idx].parent_x.copy()
            new_node.parent_y = node_list[nearest_idx].parent_y.copy()
            new_node.parent_x.append(tx)
            new_node.parent_y.append(ty)
            node_list.append(new_node)

            if dist_and_angle(tx, ty, end[0], end[1])[0] < step_size:
                path_found = True
                path = [(node.x, node.y) for node in node_list]
                move_vehicle(vehicle, path)
                return path

def get_steering_angle(vehicle_location, target_location, vehicle_transform):
            dx = target_location[0] - vehicle_location.location.x
            dy = target_location[1] - vehicle_location.location.y
            target_angle = math.atan2(dy, dx)
            vehicle_heading = math.radians(vehicle_transform.rotation.yaw)
            angle_diff = target_angle - vehicle_heading
            while angle_diff > math.pi:
                angle_diff -= 2 * math.pi
            while angle_diff < -math.pi:
                angle_diff += 2 * math.pi
            return angle_diff
def move_vehicle(self):
        if self.path:
            for waypoints in self.path:
                target_location =[waypoints[0], waypoints[1], 0]
                steering_angle = get_steering_angle(starting_point, target_location, vehicle_pos)
                max_steering_angle = math.radians(30)
                steering = max(-1.0, min(1.0, steering_angle / max_steering_angle))
                vehicle.apply_control(carla.VehicleControl(throttle=0.5, brake=0.0, steer=steering))
                time.sleep(0.1) 
  

def get_lidar_data(sensor):
    data = []

    def callback(point_cloud):
        for detection in point_cloud:
            data.append((detection.point.x, detection.point.y, 1.0))  # Assume obstacle radius is 1.0
    sensor.listen(callback)
    time.sleep(2)  # Allow sensor to gather data
    sensor.stop()
    return data

In [ ]:
def main():
   lidar_data = get_lidar_data(lidar_sensor)
   path = RRT(world, vehicle, starting_point, end, step_size, map_size, lidar_data)
   print("Path:", path)

In [ ]:
if __name__ == "__main__":
    main()